In [1]:
import glob
from re import sub
import numpy as np
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
from gensim.similarities import WordEmbeddingSimilarityIndex
from nltk.corpus import stopwords
import nltk

C:\Python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# Download stopwords
nltk.download('stopwords')

# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adamh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# Read docs
docs_dir = "ransomware"
documents = []
query = preprocess('ransomware')
stop_words = set(stopwords.words('english')) 

for i in glob.glob("%s/*.txt" % docs_dir):
    with open(i, 'r', encoding='utf-8') as f:
        documents.append(f.read())

def preprocess(doc):
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stop_words]

corpus = [preprocess(document) for document in documents]

In [26]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus + [query])
tfidf = TfidfModel(dictionary=dictionary)

In [59]:
tfidf.

In [27]:
# Create the term similarity matrix. SLOW!
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

100%|██████████| 4965/4965 [00:56<00:00, 87.72it/s] 


In [102]:
#[dictionary.doc2bow(document) for document in corpus]
a = []
for document in corpus:
    b = similarity_matrix.inner_product(dictionary.doc2bow(query), dictionary.doc2bow(document))
    a.append(b)

np.array(a).mean()

4.44

In [28]:
# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

In [29]:
doc_similarity_scores

array(0., dtype=float32)